In [1]:
import torch
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet34_Weights
import numpy as np
import torchvision.transforms as transforms        
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from src.temperature_scaling import ModelWithTemperature
from src.raps_hyp_opt import lambda_optimization_raps, k_reg_optimization

#  Reprocess
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

sorted_val_path = "D:\\Download\\ImageNet-1K\\Validation_Set\\sorted_ImageNet_val"
dataset = ImageFolder(root=sorted_val_path, transform=data_transform)

# load pre-trained model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet34(weights=ResNet34_Weights.IMAGENET1K_V1).to(device)

subset_size = len(dataset) // 10
indices = np.random.choice(len(dataset), subset_size, replace=False)
subset_dataset = Subset(dataset, indices)
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=2)
model = ModelWithTemperature(model, temperature = 1.0).to(device)
model.set_temperature(train_loader)

model.eval()

Before temperature - NLL: 1.170, ECE: 0.030
Optimal temperature: 1.015
After temperature - NLL: 1.169, ECE: 0.025


ModelWithTemperature(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True

# $\alpha$=0.1

In [2]:
lambda_values = [0, 0.01, 0.05, 0.07, 0.1, 0.2, 0.5] 
k_reg_values = [6, 7, 8, 9, 10, 11, 12] 

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_raps(model, dataset, lambda_values, k_reg = 9, device=device)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")
    
    # k_reg optimization
    print("Looking for optimal k_reg...")
    optimal_k = k_reg_optimization(model, dataset, optimal_lambda, k_reg_values, device=device)
    
    if optimal_k is None:
        print("No optimal k_reg is found")
    else:
        print(f"Optimal k_reg is {optimal_k}")

Looking for optimal lambda...
Optimal lambda is 0.07

Looking for optimal k_reg...
Optimal k_reg is 9


# $\alpha$=0.05

In [3]:
lambda_values = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3] 
k_reg_values = [7, 8, 9, 10, 11, 12, 13] 

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_raps(model, dataset, lambda_values, k_reg = 10, device=device, alpha=0.05)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")
    
    # k_reg optimization
    print("Looking for optimal k_reg...")
    optimal_k = k_reg_optimization(model, dataset, optimal_lambda, k_reg_values, device=device, alpha=0.05)
    
    if optimal_k is None:
        print("No optimal k_reg is found")
    else:
        print(f"Optimal k_reg is {optimal_k}")

Looking for optimal lambda...
Optimal lambda is 0.2

Looking for optimal k_reg...
Optimal k_reg is 9


# $\alpha$=0.2

In [4]:
lambda_values = [0.001, 0.01, 0.05, 0.1, 0.2, 0.3] 
k_reg_values = [7, 8, 9, 10, 11, 12] 

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_raps(model, dataset, lambda_values, k_reg = 10, device=device, alpha=0.2)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")
    
    # k_reg optimization
    print("Looking for optimal k_reg...")
    optimal_k = k_reg_optimization(model, dataset, optimal_lambda, k_reg_values, device=device, alpha=0.2)
    
    if optimal_k is None:
        print("No optimal k_reg is found")
    else:
        print(f"Optimal k_reg is {optimal_k}")

Looking for optimal lambda...
Optimal lambda is 0.01

Looking for optimal k_reg...
Optimal k_reg is 9
